# Run your first QML code!
## Replace the ? tokens by real code !!!

It is often helpful to see where we're going, before breaking it down into pieces, and delving into background. 
The code cells below carry out a simple instance of a quantum kernel method. 
Specifically, a single kernel matrix element is calculated. 
Users new to kernel methods or quantum kernels should not be intimidated by this; multiple lessons in this course will be devoted to dissecting exactly what is being done in these cells.

With this code we simultaneously introduce Qiskit Patterns: a framework for approaching quantum computing at the utility scale. 
This framework consists of four steps that are very general and can be applied to most problems (though in some workstreams, certain steps may be iterated multiple times).

### Qiskit patterns:

*   Step 1: Map classical inputs to a quantum problem
*   Step 2: Optimize problem for quantum execution
*   Step 3: Execute using Qiskit Runtime Primitives
*   Step 4: Analyzing / post-processing

In the cells below, we offer only cursory explanations of the various steps, just enough for you to find the appropriate lesson to learn more.

In [ ]:
# Import some qiskit packages required for setting up our quantum circuits.
from qiskit.circuit import Parameter, ParameterVector, QuantumCircuit
from qiskit.circuit.library import unitary_overlap
from qiskit import QuantumCircuit, generate_preset_pass_manager
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit.primitives import StatevectorSampler

## Step 1: Map classical inputs to a Quantum Problem

In [ ]:
# Start by getting some appropriate data. The data imported below consist of 128 rows or data points.
# Each row has 14 columns that correspond to data features, and a 15th column with a label (+/-1).
# !wget https://raw.githubusercontent.com/qiskit-community/prototype-quantum-kernel-training/main/data/dataset_graph7.csv
# The dataset can also be provided locally.

# Import some required packages, and write a function to pull some training data out of the csv file you got above.
import pandas as pd
import numpy as np


def get_training_data():
    """Read the training data."""
    df = pd.read_csv("?", sep=",", header=None)
    training_data = df.values[:20, :]
    ind = np.argsort(training_data[:, -1])
    X_train = training_data[ind][:, :-1]

    return X_train


# Prepare training data
X_train = get_training_data()

# Empty kernel matrix
num_samples = np.shape(X_train)[0]
print("Number of samples: ", num_samples)

# Prepare feature map for computing overlap between two data points.
# This could be pre-built feature maps like ZZFeatureMap, or a custom quantum circuit, as shown here.

num_features = np.shape(X_train)[1]
num_qubits = int(num_features / 2)
entangler_map = [[0, 2], [3, 4], [2, 5], [1, 4], [2, 3], [4, 6]]

fm = QuantumCircuit(num_qubits)
training_param = Parameter("θ")
feature_params = ParameterVector("x", num_qubits * 2)
fm.ry(training_param, fm.qubits)

for cz in entangler_map:
    fm.cz(cz[0], cz[1])
for i in range(num_qubits):
    fm.rz(-2 * feature_params[2 * i + 1], i)
    fm.rx(-2 * feature_params[2 * i], i)

# Pick two data points, here 14 and 19, and assign the features to the circuits as parameters.
x1 = 14
x2 = 19
unitary1 = ?.assign_parameters(list(X_train[?]) + [np.pi / 2])
unitary2 = ?.assign_parameters(list(X_train[?]) + [np.pi / 2])

# Create the overlap circuit
overlap_circ = unitary_overlap(?, ?)
?.measure_all()
?.draw("mpl", scale=0.6, style="iqp")

## Step 2: Optimize problem for Quantum Execution

In [ ]:
# Use a simulator
num_shots = 10000
sampler = StatevectorSampler()

## Step 3: Execute using Qiskit Runtime Primitives

In [ ]:
# Specify the number of shots to use.

## Evaluate the problem using statevector-based primitives from Qiskit
counts = (
    ?.run([overlap_circ], shots=?).result()[0].data.meas.get_int_counts()
)

## Step 4: Analyze and post-processing

In [ ]:
# Find the probability of 0.
0.get(0, 0.0) / num_shots

Although you don't need to understand all the steps above, we should try to understand the output, so we know why we are doing this. Many processes in machine learning use inner products as part of binary classification (among other things). Quantum mechanics has an obvious connection with this, since the probabilities of measuring various states $|\phi_i\rangle$ are given by the inner product with an initial state $|\psi\rangle$ through the inner product: $P_i = |\langle\phi_i|\psi\rangle|^2$. So what we have done above is created a quantum circuit that contains the features of our two data points, and maps them into the space of a quantum vector, then estimates the inner product in that space via making measurements. This is an example of quantum kernel estimation. Note we only implemented this process for two of the data points (the 14th and 19th). If we did this for all possible pairs, we could take the output (in this case the number 0.821...) and populate a matrix of results describing the overlap between all points in the training data set. This is the "**kernel matrix**".



#### Check your understanding

Read the question below, think about your answer, then click the triangle to reveal the solution.

<details>
  <summary>
    In the process above, we calculated a kernel matrix entry for the 14th and 19th data points. What value should we obtain if we use the same data point twice, here (like 14th and 14th again)? In other words, what should be the diagonal entries in the kernel matrix? Answer this question in the absence of noise, but note that deviations from your answer are possible in the presence of noise.
  </summary>

  **Answer:**

  The diagonals should be 1.0. This process should be calculating the normalized inner product of a vector with itself, which must always be one.
</details>



#### Developed by Eric Michiels. 